In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import re

from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, concatenate
from keras.layers import Bidirectional, LSTM
from keras.models import Model
from keras.models import load_model
from keras.callbacks import Callback

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [8]:
# 数据读取
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# read data
train = pd.read_csv('data/train.csv').fillna(' ')
test = pd.read_csv('data/test.csv').fillna(' ')
submission = pd.read_csv('data/sample_submission.csv')

# 单独保存comment_text
train_text = train['comment_text'].str.lower()
test_text = test['comment_text'].str.lower()
# 获得y_train
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

# 连接所有文字用于分词
all_text = pd.concat([train_text, test_text], axis = 0, ignore_index = True)

In [9]:
y_train.head(10)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,1,1,1,0,1,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [10]:
# 原始数据可视化分析
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
# 导入nltk预处理数据
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
# 数据预处理
# all_text = all_text[:100]
def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        elif tag.startswith('R'):
            yield wnl.lemmatize(word, pos='r')
            
        else:
            yield word

def text_cleaned_process(text_raw):    
    text_raw = str(text_raw)
    text_raw = str(text_raw.lower())
    text_raw = re.sub(r'[^a-zA-Z]', ' ', text_raw)
    
    words = text_raw.split()
    
    # 移除长度小于3的词语
    words2 = []
    for i in words:
        if len(i) >= 0:
            words2.append(i)
    # 去停止词
    stops = set(stopwords.words('english'))
    
    result_text = []
    result_text = " ".join([w for w in words2 if not w in stops])
    
    return(" ".join(lemmatize_all(result_text)))

# 去掉数字
all_text.replace({r'[^\x00-\x7F]+':''},regex=True,inplace=True)

num_all_text = all_text.size

# 输出清洗后的数据
all_text_cleaned = []

print("开始清理数据!")
for i in range(0, num_all_text):
    all_text_cleaned.append(text_cleaned_process(all_text[i]))

print("完成清理数据!")
# 构建pd形式
all_text_cleaned = pd.Series(all_text_cleaned)

开始清理数据!
完成清理数据!


In [13]:
# 原始数据可视化分析
# all_text = all_text[:100]
print("Len of all_text_cleaned:", len(all_text_cleaned))
# 数据清理前后对比
print("Text[0] before cleaned: \n", all_text[0])
print("Text[0] after cleaned: \n", all_text_cleaned[0])

Len of all_text_cleaned: 312735
Text[0] before cleaned: 
 explanation
why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27
Text[0] after cleaned: 
 explanation edits make username hardcore metallica fan revert vandalisms closure gas vote new york doll fac please remove template talk page since retire


In [14]:
# 数据分词处理
MAX_NUM_WORDS = 100000
MAX_SEQUENCE_LENGTH = 200
EMBEDDING_DIM = 300

# 分词
tokenizer = Tokenizer(num_words = MAX_NUM_WORDS)
tokenizer.fit_on_texts(all_text_cleaned)
sequences = tokenizer.texts_to_sequences(all_text_cleaned)

# 分词完成
# Pads sequences to the same length， return(len(sequence, maxlen))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)


# A dictionary of words and their uniquely assigned integers
word_index = tokenizer.word_index
print('Number of Unique Tokens',len(word_index))

# summarize what was learned
# print(tokenizer.word_counts) # A dictionary of words and their counts
# print(tokenizer.document_count) # A dictionary of words and how many documents each appeared in.
# print(tokenizer.word_docs) # An integer count of the total number of documents that were used to fit the Tokenizer.

Number of Unique Tokens 278517


In [15]:
# 重塑train与test数据
x_train = data[:len(train_text)]
x_test = data[len(train_text):]

print("Len of x_train: ", len(x_train))
print("Len of y_train: ", len(y_train))

# 拆分train数据，降低验证集数据
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.05, random_state = 255)

Len of x_train:  159571
Len of y_train:  159571


In [16]:
# 加入预训练词
EMBEDDING_FILE = 'words_vector/crawl-300d-2M.vec'

embeddings_index = {}

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

print('Total %s word vectors in pretrain data.' % len(embeddings_index))

Total 1999996 word vectors in pretrain data.


In [17]:
# 生成embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# 载入预训练词向量作为Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [22]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [26]:
# 构建CNN模型
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH, ), dtype = 'int32')
# 加入词向量
embedded_sequence = embedding_layer(sequence_input)

x = SpatialDropout1D(0.2)(embedded_sequence)

# model0
x_0 = Bidirectional(LSTM(128, return_sequences = True, dropout=0.2, recurrent_dropout=0.1))(x)
x_0 = GlobalAveragePooling1D()(x_0)

# model1
x_1 = Bidirectional(LSTM(128, return_sequences = True, dropout=0.3, recurrent_dropout=0.1))(x)
x_1 = GlobalMaxPooling1D()(x_1)

# 模型融合
x_sum = concatenate([x_0, x_1], axis = 1)
x_sum = Dense(50, activation="relu")(x_sum)
x_sum = Dropout(0.1)(x_sum)

# 压缩成对应6个标签
#conv_sum = Flatten()(conv_sum)
preds = Dense(6, activation = "sigmoid")(x_sum)

# 生成模型
model = Model(inputs = sequence_input, outputs = preds)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [27]:
# 保存模型
# model.save('LSTM_CNN_2.h5')

In [28]:
# 开始训练

# 模型加载
# model = load_model('CNN_GRU_3.h5')
# 回调函数查看训练得分
ROC_AUC = RocAucEvaluation(validation_data = (x_val, y_val), interval = 1)

history = model.fit(x_train, y_train, batch_size = 256, epochs = 5, validation_data = (x_val, y_val), verbose = 2, 
                   callbacks = [ROC_AUC])

# 计算验证集数据得分
# y_val_pred = model.predict(x_val, verbose = 2)
# score = roc_auc_score(y_val, y_val_pred)
# print("Validation data ROC-AUC score: ", score)

Train on 151592 samples, validate on 7979 samples
Epoch 1/5
 - 858s - loss: 0.0712 - acc: 0.9771 - val_loss: 0.0512 - val_acc: 0.9804

 ROC-AUC - epoch: 1 - score: 0.979072
Epoch 2/5
 - 851s - loss: 0.0485 - acc: 0.9819 - val_loss: 0.0478 - val_acc: 0.9808

 ROC-AUC - epoch: 2 - score: 0.984053
Epoch 3/5
 - 851s - loss: 0.0450 - acc: 0.9827 - val_loss: 0.0464 - val_acc: 0.9810

 ROC-AUC - epoch: 3 - score: 0.985836
Epoch 4/5
 - 851s - loss: 0.0432 - acc: 0.9833 - val_loss: 0.0458 - val_acc: 0.9816

 ROC-AUC - epoch: 4 - score: 0.986399
Epoch 5/5
 - 851s - loss: 0.0419 - acc: 0.9837 - val_loss: 0.0447 - val_acc: 0.9820

 ROC-AUC - epoch: 5 - score: 0.987048


In [29]:
submission = pd.read_csv('data/sample_submission.csv')

In [32]:
# test数据预测值
y_prediction = model.predict(x_test, batch_size =1024, verbose = 1)

# 生成语言各分类概率
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_prediction

# 输出submission.csv
submission.to_csv('submission_lstm_fasttex.csv', index=False)

153164/153164 [==============================] - 134s 872us/step


In [33]:
# 初步检查输出结果
submission.head(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.997645,4.509615e-01,0.983210,0.167452,0.914684,0.380847
1,0000247867823ef7,0.000671,7.268186e-07,0.000062,0.000008,0.000077,0.000006
2,00013b17ad220c46,0.000904,3.388999e-06,0.000143,0.000036,0.000168,0.000041
3,00017563c3f7919a,0.000123,1.509362e-07,0.000015,0.000004,0.000016,0.000002
4,00017695ad8997eb,0.002842,5.976052e-06,0.000170,0.000049,0.000227,0.000030
5,0001ea8717f6de06,0.000115,1.015672e-07,0.000009,0.000004,0.000015,0.000002
6,00024115d4cbde0f,0.002264,2.330638e-06,0.000318,0.000023,0.000351,0.000016
7,000247e83dcc1211,0.450074,1.516326e-03,0.026319,0.001539,0.103240,0.002929
8,00025358d4737918,0.259050,2.727079e-04,0.009522,0.001149,0.090293,0.000637
9,00026d1092fe71cc,0.000476,3.525271e-07,0.000042,0.000005,0.000045,0.000004
